<a href="https://colab.research.google.com/github/Haiz14/rag_practice/blob/main/simple_rag_gemini1.5_and_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Add GOOGLE_API_KEY in secrets for gemini llm

Simple rag, create download data, create vector emeddings and call gemini for response.


In [8]:
from google.colab import userdata
api_key = userdata.get("GOOGLE_API_KEY")

In [1]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-embeddings-instructor llama-index-llms-gemini

In [2]:
# add data folder with few pdf
!mkdir data
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt -O data/state_of_the_union.txt

mkdir: cannot create directory ‘data’: File exists
--2025-03-18 08:39:12--  https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘data/state_of_the_union.txt’

data/state_of_the_u 100%[===================>]  38.11K  --.-KB/s    in 0.01s   

2025-03-18 08:39:12 (3.38 MB/s) - ‘data/state_of_the_union.txt’ saved [39027/39027]



In [11]:
# setup llamaindex starter
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
documents = SimpleDirectoryReader('./data').load_data()
# set settings to use local model
index = VectorStoreIndex.from_documents(documents)

# New Section

In [12]:
# test retriever
retriever = index.as_retriever()
retriever.retrieve("What did the president say about Ketanji Brown Jackson")

[NodeWithScore(node=TextNode(id_='1b80c8d1-b553-43ca-95c8-0da07cd6091a', embedding=None, metadata={'file_path': '/content/data/state_of_the_union.txt', 'file_name': 'state_of_the_union.txt', 'file_type': 'text/plain', 'file_size': 39027, 'creation_date': '2025-03-18', 'last_modified_date': '2025-03-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2bd671cd-fab6-4aca-87ef-a90097d830a7', node_type='4', metadata={'file_path': '/content/data/state_of_the_union.txt', 'file_name': 'state_of_the_union.txt', 'file_type': 'text/plain', 'file_size': 39027, 'creation_date': '2025-03-18', 'last_modified_date': '2025-03-18'}, hash='88c9db43ad79ec1d492bb110c07fea8119b3b01b25a60fca7b444076e3864b72'), <NodeRelationship

In [14]:
# set up a query engine
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

# Option 2: return a string (we use a raw LLM call for illustration)

from llama_index.llms.gemini import Gemini
from llama_index.core import PromptTemplate

qa_prompt = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: Gemini
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)

In [15]:
# test a response from above engine, use gemini as llm
from llama_index.llms.gemini import Gemini

model = Gemini(
    model = "models/gemini-1.5-flash",
    api_key = api_key,
    streaming = False
)
synthesizer = get_response_synthesizer(model)
query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=synthesizer,
    llm=model,
    qa_prompt=qa_prompt,
)

The provided text does not mention Ketanji Brown Jackson.



In [16]:
response = query_engine.custom_query("What does the data say")
print(response)

The provided text is a speech, likely the State of the Union address.  It covers several key topics:

* **The State of the Union:** The speaker declares the state of the union strong, emphasizing national resilience and the ability to overcome challenges.  They express optimism about the future.

* **COVID-19:** The speaker calls for a bipartisan approach to addressing the pandemic, urging a shift away from partisan division.

* **Crime and Policing:** The speech addresses the deaths of two police officers, emphasizing the need for investment in crime prevention, community policing, and police training.  It explicitly rejects "defunding the police," advocating for increased funding and resources.

* **Gun Violence:**  The speaker calls for stricter gun control measures, including universal background checks, banning assault weapons and high-capacity magazines, and repealing the liability shield for gun manufacturers.

* **Voting Rights:** The speech highlights concerns about voting rig